## Tutorial: Using pd_exp to create experiments with Prisoner's Dilemma Tournaments
This notebook walks through two different experimental setups to showcase the code functionality of the `pd_exp` module.

In [1]:
# Local modules to import
import pd_exp
from helper_funcs import partitions
from settings import player_names, stag

# Other Python modules used
import pandas as pd
from pathlib import Path
import time, json

`pd_exp` was designed to collect data from multiple groups of round-robin tournaments. To keep vocabulary consistent, let me introduce some project terminology:
- _Supergame_: a single PD instance between two players that consists of one or more rounds of play.
- _Tournament_: a round-robin tournament between a select group of players. This concept can also be referred to as a team.
- _System_: a collection of one or more teams.

In the code below, I setup data collection for two different experiments. In experiment 1, we consider how measured outcomes differ within one team of deterministic players in two different game variations: classic PD and Stag Hunt. In experiment 2, we expand on experiment 1 to consider multiple different systems, where each system is comprised of two teams.

#### __Team Metrics__
To measure team outcomes, we use the following metrics: 1) minimum of normalized player scores, 2) average of normalized player scores, and 3) average normalized CC distribution.

##### __The Minumum (Min Score) and Average (Avg Score) of Normalized Player Scores__
The normalized score for a player, $j$, is computed as 
$$
S_j = \sum_{i\neq j} \frac{\text{Score-per-turn against opponent } i}{\text{Total opponents}} = \frac{1}{\text{Total opponents}} \sum_{i\neq j} \frac{S_{ji}}{T_{ji}},
$$
where $S_{ji}$ is player $j$'s score against player $i$ and $T_{ji}$ is the total turns played between player $j$ and $i$.

The minimum and average values across all normalized player scores are taken as aggregate group measures to reflect group welfare.

##### __The Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{1}{N} \sum_{j=1}^N \frac{1}{N-1} \sum_{i \neq j} \frac{CC \text{ count against player } i}{\text{total turns against player }i}
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.

##### __The New Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{1}{N} \sum_{j=1}^N \sum_{i \neq j} \frac{CC \text{ count against player } i}{\text{total turns against player }i}
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.

##### __The New Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{1}{N} \sum_{j=1}^N \text{ Total Score in the Tournament }
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.

#### __System Metrics__
To measure system outcomes, we extend the above metrics and use six different measures:
1. System Minimum Score (SYS MIN)
2. System Average Score (SYS AVG)
3. Minimum of Team Average Scores (Min of Team Avgs)
4. Average of Team Minimum Scores (Avg of Team Mins)
5. System CC Distribution Average (SYS CC Dist AVG)
6. System CC Distribution Average (SYS CC Dist MIN)

### Experiment 1: Comparing team performance across classic PD and Stag Hunt

In [2]:
### For experiment 1
# Since this simulation gathers data for systems, I coded systems as ordered-lists of 
# teams. To implement ordered-lists of teams I nested player_names within closed-brackets and then a tuple to create an ordered-list.
# The simulation code allows for processing numerous systems, so I enclose the tournament within another pair
# of closed-brackets and then a tuple.

part_list = tuple([tuple([player_names])]) # 'part' is shorthand for partition

In [3]:
print(part_list)

((['Alternator', 'Anti Tit For Tat', 'Bully', 'Cooperator', 'Cycler DC', 'Defector', 'Suspicious Tit For Tat', 'Tit For Tat', 'Win-Shift Lose-Stay', 'Win-Stay Lose-Shift'],),)


In [4]:
print('Total count of systems: ',len(part_list))  #Should be 5775 for n=12 and k=4
start_time = time.time()

print('Instantiate PD Experiments with different game types')
# To instantiate a PD Experiment, I use the PDExp constructor with at partition list and game-type, which is 
# by default the classic PD)
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)


print('Running experiments and computing data')
# To generate the simulation data and compute the group metrics, I use the run_experiments() method.
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()


print('Saving experiment data')
### For experiment 1
path = 'Data/Experiment1/'
classic_pdExp.save_data(path, 'ClassicPD_test_DELETE')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_test_DELETE')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  1
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  (['Alternator', 'Anti Tit For Tat', 'Bully', 'Cooperator', 'Cycler DC', 'Defector', 'Suspicious Tit For Tat', 'Tit For Tat', 'Win-Shift Lose-Stay', 'Win-Stay Lose-Shift'],)
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift
Analysing: 100%|██████████| 25/25 [00:00<00:00, 82.24it/s]***Processing number  1
partition list:  (['Alternator', 'Anti Tit For Tat', 'Bully', 'Cooperator', 'Cycler DC', 'Defector', 'Suspicious Tit For Tat', 'Tit For Tat', 'Win-Shift Lose-Stay', 'Win-Stay Lose-Shift'],)
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift

Analysing: 100%|██████████| 25/25 

### Experiment 1: Data Comparison Across Games

In [5]:
# Retrieve saved data
classic_dataf1 = pd.read_csv('Data/Experiment1/ClassicPD_test_DELETE_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf1 = pd.read_csv('Data/Experiment1/StagHunt_test_DELETE_RPST_5_1_0_3.csv', index_col=0)

#### _The System ID_
Since these experiments are with unique players, each system is denoted with a unique ID that is comprised of numbers, commas, and underscores. The numbers represent players and determined by the alphabetical order of the player's strategy out of the total set of player strategies in system. The commas delimit players of the same team, and underscores delimit different teams of the same system. Note the system IDs and team compositions below and how they differ between experiment 1 and experiment 2. 

In [6]:
# I use option_context to customize the limits of the dataframe display
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,6,7,8,9,10","Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.284385,1.409722,0.248897


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,6,7,8,9,10","Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.257786,1.994048,0.248897


In [8]:
# SIDENOTE: Documentation (i.e. docstring) for code can be retrieved with '?'. Here are some examples.

?pd_exp

Type:        module
String form: <module 'pd_exp' from '/home/s/PrisonersDilemma-main/pd_exp.py'>
File:        ~/PrisonersDilemma-main/pd_exp.py
Docstring:  
Pd_exp: Prisoner's Dilemma Experimentation

Generate data for numerous Prisoner's Dilemma tournaments or systems of 
tournaments.

Classes:

    PdTournament
        A class to represent a tournament.
    PdSystem 
        Generate data for multiple tournaments and organize it into a dataframe
    PdExp
        Generate data for multiple systems
        
Functions:

    grouper(iterable, n, fillvalue=None) -> iterable of n-sized-chunks
        Collect data into fixed-length chunks or blocks
    avg_normalised_state(object, tuple) -> float
        Returns the tournament average for given state distribution (e.g.
        (C,C), (D,D), (C,D), (D,C))


In [9]:
?pd_exp.PdExp

Init signature: pd_exp.PdExp(tuple_of_systems, game_type=None)
Docstring:     
A class to represent a series of systems. 

...

Attributes
----------
sys_tuple : tuple
    three-dimensional ordered list, or tuple, where the first-dimension is
    a list of systems, the 2nd-dimension is the team list, and the 
    3rd-dimension is the individual players.
game : axelrod.game (object)
    container for game matrix and scoring logic 

Methods
-------
run_experiments:
    Iterates through each system, runs the different tournaments and 
    then saves the data to the data attribute.
save_data(path_to_directory, descrip_name):
    Saves experiment data as a csv file
Init docstring:
Constructs all the necessary attributes for pd experiment object

Parameters
----------
tuple_of_systems : tuple
    a multi-dimensional matrix where each item of the first-dimension is an
    ordered list of systems, and the second dimension is an ordered-list of 
    teams
game_type : axelrod.game (object)
    c

### Experiment 2: Comparing systems of 3 teams across classic PD and Stag Hunt

In [10]:
### For experiment 2
# To create numerous systems with a partitioned list of teams can take time, so I save the tuple of systems 
# to file immediately after computing all k-partition subsets of a set of players  to avoid having to recompute 
# them at each run.

# Checking for txt file with list of systems
p = Path('partition_list_teams_of5_DEMO.txt')
if p.exists():
   with open(p, "r") as f:
       part_list = json.load(f)
else:
   print('creating partitions and saving to file')
   part_list = list(partitions(player_names,5))
   p.touch()
   with open(p, "w") as f:
       json.dump(part_list, f)

In [11]:
part_list = part_list[:3] # Let's just consider 3 different systems
print('Total count of systems: ',len(part_list))
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  3
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [['Alternator', 'Cycler DC', 'Anti Tit For Tat', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Defector']]
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Cycler DC,Suspicious Tit For Tat,Win-Stay Lose-Shift
Analysing: 100%|██████████| 25/25 [00:00<00:00, 95.23it/s]Instantiating tournament object with these players:  Bully,Cooperator,Defector,Tit For Tat,Win-Shift Lose-Stay

Analysing: 100%|██████████| 25/25 [00:00<00:00, 112.58it/s]***Processing number  1
partition list:  [['Cycler DC', 'Anti Tit For Tat', 'Defector', 'Alternator', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Win-Stay Lose-Shift']]
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Cycler DC,Defector,Suspicious T

### Experiment 2: Data Comparison Across Systems, Teams, and Games

In [12]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf = pd.read_csv('Data/Experiment2/StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)
stagHunt_dataf = stagHunt_dataf.reset_index(drop=True)

In [13]:
# The computed results create a large spreadsheet or dataframe. Here is a list of just the column names.
classic_dataf.columns

Index(['System ID', 'SYS MIN Score', 'SYS AVG Score', 'MIN of Team Avgs',
       'AVG of Team Mins', 'SYS CC Dist AVG', 'SYS CC Dist MIN',
       'SYS New CC Dist AVG', 'SYS New CC Dist MIN', 'Team1', 'Player1',
       'P1_Norm_Score', 'Player2', 'P2_Norm_Score', 'Player3', 'P3_Norm_Score',
       'Player4', 'P4_Norm_Score', 'Player5', 'P5_Norm_Score',
       'Team1 Avg Score', 'Team1 New Avg Score', 'Team1 Min Score',
       'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2', 'Player1.1',
       'P1_Norm_Score.1', 'Player2.1', 'P2_Norm_Score.1', 'Player3.1',
       'P3_Norm_Score.1', 'Player4.1', 'P4_Norm_Score.1', 'Player5.1',
       'P5_Norm_Score.1', 'Team2 Avg Score', 'Team2 New Avg Score',
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist'],
      dtype='object')

In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team2']])

,System ID,Team1,Team2
0,"1,2,5,7,10_3,4,6,8,9","Alternator,Anti Tit For Tat,Cycler DC,Suspicious Tit For Tat,Win-Stay Lose-Shift","Bully,Cooperator,Defector,Tit For Tat,Win-Shift Lose-Stay"
1,"1,2,5,6,7_3,4,8,9,10","Alternator,Anti Tit For Tat,Cycler DC,Defector,Suspicious Tit For Tat","Bully,Cooperator,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift"
2,"1,2,6,7,10_3,4,5,8,9","Alternator,Anti Tit For Tat,Defector,Suspicious Tit For Tat,Win-Stay Lose-Shift","Bully,Cooperator,Cycler DC,Tit For Tat,Win-Shift Lose-Stay"


In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team2']])

,System ID,Team1,Team2
0,"1,2,5,7,10_3,4,6,8,9","Alternator,Anti Tit For Tat,Cycler DC,Suspicious Tit For Tat,Win-Stay Lose-Shift","Bully,Cooperator,Defector,Tit For Tat,Win-Shift Lose-Stay"
1,"1,2,5,6,7_3,4,8,9,10","Alternator,Anti Tit For Tat,Cycler DC,Defector,Suspicious Tit For Tat","Bully,Cooperator,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift"
2,"1,2,6,7,10_3,4,5,8,9","Alternator,Anti Tit For Tat,Defector,Suspicious Tit For Tat,Win-Stay Lose-Shift","Bully,Cooperator,Cycler DC,Tit For Tat,Win-Shift Lose-Stay"


#### How do the systems compare within the same game?

In [16]:

classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"1,2,5,7,10_3,4,6,8,9",1.107143,2.271399,2.236726,1.589286,0.214762,0.208571,2.841667,2.64
1,"1,2,5,6,7_3,4,8,9,10",1.500000,2.342113,2.100000,1.678571,0.287976,0.125000,3.316667,1.75
2,"1,2,6,7,10_3,4,5,8,9",1.375000,2.264851,2.078571,1.473214,0.242024,0.153571,2.980000,2.15


In [17]:

stagHunt_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"1,2,5,7,10_3,4,6,8,9",1.739583,2.139673,2.130595,1.753125,0.214762,0.208571,2.841667,2.64
1,"1,2,5,6,7_3,4,8,9,10",1.482143,2.407292,1.783333,1.953869,0.287976,0.125000,3.316667,1.75
2,"1,2,6,7,10_3,4,5,8,9",1.520833,2.228363,1.871429,1.906250,0.242024,0.153571,2.980000,2.15


#### How did team 1 and 2 performance differ across systems?

In [21]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist,Team2,Team2 Avg Score,Team2 New Avg Score,Team2 Min Score,Team2 Avg CC Dist,Team2 New Avg CC Dist
0,"1,2,5,7,10_3,4,6,8,9","Alternator,Anti Tit For Tat,Cycler DC,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.306071,85.6,2.071429,0.208571,2.64,"Bully,Cooperator,Defector,Tit For Tat,Win-Shift Lose-Stay",2.236726,84.2,1.107143,0.220952,3.043333
1,"1,2,5,6,7_3,4,8,9,10","Alternator,Anti Tit For Tat,Cycler DC,Defector,Suspicious Tit For Tat",2.100000,85.4,1.500000,0.125000,1.75,"Bully,Cooperator,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.584226,94.0,1.857143,0.450952,4.883333
2,"1,2,6,7,10_3,4,5,8,9","Alternator,Anti Tit For Tat,Defector,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.078571,83.6,1.571429,0.153571,2.15,"Bully,Cooperator,Cycler DC,Tit For Tat,Win-Shift Lose-Stay",2.451131,90.6,1.375000,0.330476,3.810000


In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
                       'display.max_colwidth', None):  # more options can be 
                                                       # specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist,Team2,Team2 Avg Score,Team2 New Avg Score,Team2 Min Score,Team2 Avg CC Dist,Team2 New Avg CC Dist
0,"1,2,5,7,10_3,4,6,8,9","Alternator,Anti Tit For Tat,Cycler DC,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.130595,92.8,1.766667,0.208571,2.64,"Bully,Cooperator,Defector,Tit For Tat,Win-Shift Lose-Stay",2.148750,83.0,1.739583,0.220952,3.043333
1,"1,2,5,6,7_3,4,8,9,10","Alternator,Anti Tit For Tat,Cycler DC,Defector,Suspicious Tit For Tat",1.783333,59.4,1.482143,0.125000,1.75,"Bully,Cooperator,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",3.031250,99.6,2.425595,0.450952,4.883333
2,"1,2,6,7,10_3,4,5,8,9","Alternator,Anti Tit For Tat,Defector,Suspicious Tit For Tat,Win-Stay Lose-Shift",1.871429,66.8,1.520833,0.153571,2.15,"Bully,Cooperator,Cycler DC,Tit For Tat,Win-Shift Lose-Stay",2.585298,91.8,2.291667,0.330476,3.810000
